In [240]:
import requests
import numpy as np
import pandas as pd

In [241]:
#Currently this has been tested on all 5 of the different sets in Standard and it
#works fine with those.  More testing soon!

In [242]:
def removePunctuationFromList(listPlz):
    '''this function removes characters not supported by mtggoldfish.com.  As a note, they do support dashes.
    This function may not be 100% accurate as of now due to lack of testing symbols.'''
    lisk = listPlz[:]
    punctuation = [".",",","'",'"',"?","!","/","%","$","#","@","^","&","(",")","_","+","*"]
    for i in range(len(lisk)):
        for p in punctuation:
            lisk[i] = lisk[i].replace(p,"")
    return lisk

In [243]:
def spacesToPlus(listPlz):
    '''The URLS on mtggoldfish.com require plus-separated names for cards instead of space-separated ones.
    Also, this function handles the weird cards that appear as Word1 // Word2 (Word#) on gatherer.wizards.com'''
    lisk = listPlz[:]
    #replacing regular spaces with +
    for i in range(len(lisk)):
        lisk[i] = lisk[i].replace(" ","+")
        
    # handling double plus exception    
    for i in range(len(lisk)):
        if "++" in lisk[i]:
            lisk[i] = lisk[i].split("+")[0]+"+"+lisk[i].split("+")[2]
    
    # handling other part of double plus exception where the card name occurs twice.
    removeList = []
    for i in range(len(lisk)):
        if lisk.count(lisk[i]) == 2 and lisk[i] not in removeList:
            removeList.append(lisk[i])
    for i in removeList:
        lisk.remove(i)
    return lisk

In [244]:
def getRareAndMythicNames(setName):
    '''this function takes one parameter, the name of the set you would like to evaluate booster box value for.
    After you input the name of the set, it will grab all the names of the rares and mythics from
    gatherer.wizards.com and output them into 2 lists, rareNames and mythicNames.'''
    
    #If their website is updated, these queries may not work and would need to be altered.
    
    #setName = "Ixalan"
    rmList = ["R","M"]
    numCards = 0
    cardNames = [[],[]]
    for i in rmList:
        URL = "http://gatherer.wizards.com/Pages/Search/Default.aspx?action=advanced&set=+[%22"+setName +"%22]&rarity=+["+ i + "]"
        r = requests.get(URL)
        HTML = r.text
        numCards = int(HTML.split('searchTermDisplay')[3].split("(")[1].split(')')[0])
        
        for x in range(numCards):
            tempName = HTML.split('</a></span> <span class="manaCost">')[x].split('>')[-1]
            #print(tempName)
            cardNames[rmList.index(i)].append(tempName)
        #print(cardNames)
    #rareNames,mythicNames --> out -- these are for searching on mtggoldfish.com and will be altered.
    rareNames = cardNames[0]
    mythicNames = cardNames[1]
    
    #these are going to be the unaltered real names of the cards.
    #rareList = cardNames[0][:]
    #mythicList = cardNames[1][:]
    
    #removing punctuation
    rareNames = removePunctuationFromList(rareNames)
    mythicNames = removePunctuationFromList(mythicNames)
    
    #replacing " " with "+"
    rareNames = spacesToPlus(rareNames)
    mythicNames = spacesToPlus(mythicNames)
    
    #the first two here are altered name lists.  The last two are the unaltered, real names of the cards.
    return rareNames, mythicNames#, rareList, mythicList

In [245]:
def getCurrentCardValues(listOfCardNames,setName):
    '''generalized function to grab all the current card prices for a list of card names.'''
    cardPrices = []
    for i in range(len(listOfCardNames)):
        URL = 'https://www.mtggoldfish.com/price/'+setName+'/'+listOfCardNames[i]
        r = requests.get(URL)
        HTML = r.text
        cardPrices.append(HTML.split("<div class='price-box-type'>PAPER</div>")[1].split('price-box-price')[1].split('>')[1].split('<')[0])
        #print(HTML.split("<div class='price-box-type'>PAPER</div>")[1].split('price-box-price')[1].split('>')[1].split('<')[0])
    return cardPrices

In [246]:
def setScraper(setName):
    '''The main function! Enter a set name!
    
    This function will use all the above functions to pull an entire set of mtg card names
    from gatherer.wizards.com, and then pull all the current prices from mtggoldfish.com, and output all of it
    neatly into a pandas DataFrame for you!'''
    rareNames, mythicNames = getRareAndMythicNames(setName)
    #print(rareNames)
    #print(mythicNames)
    alteredSetName = setName.replace(" ","+")
    #print(alteredSetName)
    rarePrices = getCurrentCardValues(rareNames,alteredSetName)
    mythicPrices = getCurrentCardValues(mythicNames,alteredSetName)
    
    for i in range(len(rarePrices)):
        rarePrices[i] = float(rarePrices[i])
        
    for i in range(len(mythicPrices)):
        mythicPrices[i] = float(mythicPrices[i])
    
    rareLabeler = ["Rare"]*len(rareNames)
    mythicLabeler = ["Mythic"]*len(mythicNames)
    
    df = pd.DataFrame()
    df["cardName"] = rareNames+mythicNames
    df["rarity"] = rareLabeler+mythicLabeler
    df["currentPrice"] = rarePrices+mythicPrices
    
    return df

In [247]:
def averages(df):
    '''function that returns the average rare and mythic values for the set. Once you run setScraper and have a
    df saved as a variable, throw it in here to get averages.'''
    averageRareValue = np.mean(df[df["rarity"]=="Rare"]["currentPrice"].values)
    averageMythicValue = np.mean(df[df["rarity"]=="Mythic"]["currentPrice"].values)
    return averageRareValue, averageMythicValue

In [248]:
def safeBoxValueEstimate(averageMythicsPerBox,averageRareValue,averageMythicValue,packsPerBox=36):
    '''A function that takes the average value of the rares and mythics in a set, presumes 36 packs per box,
    and asks for the average mythics per box for the set.  In Ixalan, for example, there are said to be less
    mythics per box.  This is the one bit of outside knowledge you will need to have in order to
    properly use this function.'''
    mythics = averageMythicsPerBox
    rares = packsPerBox - mythics
    estimate = 0
    estimate += (averageRareValue*rares)
    estimate += (averageMythicValue*mythics)
    return estimate

In [249]:
def setSummary(df,averageMythicsPerBox,packsPerBox=36):
    ''' Main summary statistics for a set!
    
    A function that takes the average value of the rares and mythics in a set, presumes 36 packs per box,
    and asks for the average mythics per box for the set.  In Ixalan, for example, there are said to be less
    mythics per box.  This is the one bit of outside knowledge you will need to have in order to
    properly use this function.  This one prints the averages and a conservative estimate for the average
    value of a typical box, again presuming you're right about the average mythic count per box.'''
    
    averageRareValue, averageMythicValue = averages(df)
    print("averageRareValue: ","$"+str(averageRareValue))
    print("averageMythicValue: ","$"+str(averageMythicValue))
    estimate = safeBoxValueEstimate(averageMythicsPerBox,averageRareValue,averageMythicValue)
    print("estimated box value: ", "$"+str(estimate))

In [187]:
Ixalan = setScraper("Ixalan")

In [273]:
Ixalan.head()

cardName rarity  currentPrice
0  Adanto, the First Fort   Rare          3.23
1     Angrath's Marauders   Rare          0.31
2       Arcane Adaptation   Rare          0.61
3     Arguel's Blood Fast   Rare          1.02
4  Ashes of the Abhorrent   Rare          0.76

In [200]:
setSummary(Ixalan,4) # <--- second input is average mythics per box, this is outside knowledge you need to look up!
#currently, 4 is a placeholder because mythics are supposed to be slightly less common in Ixalan, but I don't have
# the real number right now.

averageRareValue:  $2.57066666667
averageMythicValue:  $6.19647058824
estimated box value:  $107.047215686


In [202]:
Kaladesh = setScraper("Kaladesh")

In [203]:
Kaladesh.head()

cardName rarity  currentPrice
0      Aetherflux Reservoir   Rare          0.82
1      Aethersquall Ancient   Rare          0.27
2           Aetherstorm Roc   Rare          0.26
3          Animation Module   Rare          0.49
4  Architect of the Untamed   Rare          0.29

In [207]:
setSummary(Kaladesh,4.5) #Assuming 1/8 packs have Mythics

averageRareValue:  $1.34763636364
averageMythicValue:  $6.05470588235
estimated box value:  $69.6967219251


In [251]:
Amonkhet = setScraper("Amonkhet")

In [252]:
Amonkhet.head()

cardName rarity  currentPrice
0         Anointed+Procession   Rare          7.95
1  Approach+of+the+Second+Sun   Rare          3.48
2          Archfiend+of+Ifnir   Rare          0.63
3             Aven+Mindcensor   Rare          0.67
4          Bounty+of+the+Luxa   Rare          0.41

In [253]:
setSummary(Amonkhet,4.5)

averageRareValue:  $1.49327272727
averageMythicValue:  $5.60941176471
estimated box value:  $72.2804438503


In [260]:
Hour = setScraper("Hour of Devastation")

In [257]:
Hour.head()

cardName rarity  currentPrice
0  Abandoned+Sarcophagus   Rare          0.38
1        Adorned+Pouncer   Rare          1.21
2          Ammit+Eternal   Rare          1.44
3  Angel+of+Condemnation   Rare          0.43
4       Apocalypse+Demon   Rare          0.32

In [259]:
setSummary(Hour,4.5)

averageRareValue:  $1.06954545455
averageMythicValue:  $7.34285714286
estimated box value:  $66.733538961


In [262]:
Aether = setScraper("Aether Revolt")

In [263]:
Aether.head()

cardName rarity  currentPrice
0       Aethergeode+Miner   Rare          0.35
1  Aethersphere+Harvester   Rare          2.34
2        Aethertide+Whale   Rare          0.30
3       Aid+from+the+Cowl   Rare          0.29
4              Ajanis+Aid   Rare          0.31

In [264]:
setSummary(Aether,4.5)

averageRareValue:  $1.64272727273
averageMythicValue:  $3.68785714286
estimated box value:  $68.3412662338


### Now it's worth noting that 2 of these sets (Hour of Devastation and Amonkhet) have an exception that hasn't been accounted for (more research needs to be done.)  They are the ONLY sets to ever have very very rare cards called Invocations that you can sell for A LOT.  I can't include them right now though because I don't know how often they occur.

### The safe bet though (in Standard sets) is Ixalan.  I'm not even accounting for holographic cards or uncommon cards and it is well above the purchase price of 88 at a whopping 107.

In [267]:
Eldritch = setScraper("Eldritch Moon")

In [270]:
setSummary(Eldritch,4.5)

averageRareValue:  $1.0048
averageMythicValue:  $5.0
estimated box value:  $54.1512


### Wow, Eldritch Moon boxes are going for 82 dollars online. What a ripoff!!